In [1]:
import pandas as pd
# import keras

In [2]:
deliv_train = pd.read_csv('TrainDeliveries.csv')
deliv_train.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Team1,Team5,1,1,Player 105,Player 354,Player 431,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Team1,Team5,1,2,Player 105,Player 354,Player 431,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Team1,Team5,1,3,Player 105,Player 354,Player 431,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Team1,Team5,1,4,Player 105,Player 354,Player 431,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Team1,Team5,1,5,Player 105,Player 354,Player 431,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [3]:
match_train = pd.read_csv('Trainmatches.csv',index_col=0)
match_train.head()
match_test = pd.read_csv('Testmatches.csv')
match_test.head()

,match_id,season,city,team1,team2,toss_winner,toss_decision,result,dl_applied,venue
0,501,2014,City22,Team2,Team9,Team2,bat,normal,0,Stadium23
1,502,2014,City9,Team7,Team8,Team8,field,normal,0,Stadium8
2,503,2014,City1,Team5,Team1,Team5,bat,normal,0,Stadium1
3,504,2014,City8,Team10,Team6,Team6,field,normal,0,Stadium7
4,505,2014,City10,Team8,Team2,Team2,field,normal,0,stadium35


In [4]:
deliv_test = pd.read_csv('TestDeliveries.csv')
deliv_test.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,501,1,Team2,Team9,1,1,Player 260,Player 234,Player 48,0,...,0,0,0,0,1,0,1,NaN,NaN,NaN
1,501,1,Team2,Team9,1,2,Player 234,Player 260,Player 48,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,501,1,Team2,Team9,1,3,Player 234,Player 260,Player 48,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
3,501,1,Team2,Team9,1,4,Player 234,Player 260,Player 48,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,501,1,Team2,Team9,1,5,Player 234,Player 260,Player 48,0,...,0,0,0,0,1,0,1,NaN,NaN,NaN


In [5]:
uniq_ids = match_test.match_id.unique()

In [6]:
sub = pd.DataFrame(uniq_ids)
sub.columns = ["match_id"]
sub.head()

,match_id
0,501
1,502
2,503
3,504
4,505


In [7]:
innings = [1,2]
sub = match_test[["match_id","team1"]]
sub.head()

,match_id,team1
0,501,Team2
1,502,Team7
2,503,Team5
3,504,Team10
4,505,Team8


In [8]:
sub.loc[sub["match_id"] == 501]
sub[sub["match_id"] == 501]["team1"].tolist()[0]

'Team2'

In [9]:
sum(deliv_test[(deliv_test["match_id"] == 501) & (deliv_test["inning"] == 1)]["total_runs"])

178

In [10]:
batting_table = pd.DataFrame(columns=['match_id','innings','batting_team','inning_score'])
print(batting_table)

Empty DataFrame
Columns: [match_id, innings, batting_team, inning_score]
Index: []


In [11]:
innings = [1,2]
entries = []
for idx in sub["match_id"]:
    for inning in innings:
        try:
            bat = (deliv_test[(deliv_test["match_id"] == idx) & (deliv_test["inning"] == inning)]["batting_team"].unique().tolist())[0]
        except IndexError:
            continue
        entries.append([idx,inning,bat,sum(deliv_test[(deliv_test["match_id"] == idx) & (deliv_test["inning"] == inning)]["total_runs"])])
       
       # batting_table["inning_score"] = sum(deliv_test[(deliv_test["match_id"] == idx) & (deliv_test["inning"] == inning)]["total_runs"])
    
batting_table = pd.DataFrame(entries,columns=['match_id','innings','batting_team','inning_score'])
batting_table.head()

,match_id,innings,batting_team,inning_score
0,501,1,Team2,178
1,501,2,Team9,153
2,502,1,Team7,164
3,502,2,Team8,165
4,503,1,Team5,160


In [15]:
for idx in sub["match_id"]:
    try:
        in1_score = batting_table[(batting_table["match_id"] == idx) & (batting_table["innings"] == innings[0])]["inning_score"].tolist()[0]
        in2_score = batting_table[(batting_table["match_id"] == idx) & (batting_table["innings"] == innings[1])]["inning_score"].tolist()[0]
        if in1_score > in2_score:
            winning = batting_table[(batting_table["match_id"] == idx) & (batting_table["innings"] == innings[0])]["batting_team"].tolist()[0]
            if sub[sub["match_id"] == idx]["team1"].tolist()[0] == winning:
                sub.loc[sub["match_id"] == idx,"team_1_win_flag"] = 1
            else:
                sub.loc[sub["match_id"] == idx,"team_1_win_flag"] = 0
        elif in1_score < in2_score:
            winning = batting_table[(batting_table["match_id"] == idx) & (batting_table["innings"] == innings[1])]["batting_team"].tolist()[0]
            if sub[sub["match_id"] == idx]["team1"].tolist()[0] == winning:
                sub.loc[sub["match_id"] == idx,"team_1_win_flag"] = 1
            else:
                sub.loc[sub["match_id"] == idx,"team_1_win_flag"] = 0
        
        else:
            sub.loc[sub["match_id"] == idx,"team_1_win_flag"] = 0
    except IndexError:
        sub.loc[sub["match_id"] == idx,"team_1_win_flag"] = 0
        continue

/home/shankar/.conda/envs/ml/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [16]:
print(sub)

     match_id   team1  team_1_win_flag
0         501   Team2              1.0
1         502   Team7              0.0
2         503   Team5              0.0
3         504  Team10              0.0
4         505   Team8              0.0
5         506   Team6              1.0
6         507  Team10              0.0
7         508   Team2              1.0
8         509   Team8              1.0
9         510   Team5              0.0
10        511   Team1              0.0
11        512   Team7              0.0
12        513   Team9              0.0
13        514   Team6              1.0
14        515   Team2              0.0
15        516   Team8              1.0
16        517   Team8              0.0
17        518   Team2              0.0
18        519  Team10              1.0
19        520   Team9              1.0
20        521  Team10              1.0
21        522   Team6              0.0
22        523   Team7              0.0
23        524   Team8              1.0
24        525   Team5    

In [18]:
print(sub.isnull().any().any())

False


In [19]:
final_sub = sub[['match_id','team_1_win_flag']]
final_sub.head()

,match_id,team_1_win_flag
0,501,1.0
1,502,0.0
2,503,0.0
3,504,0.0
4,505,0.0


In [20]:
final_sub.to_csv('AAsub.csv',index=False)